<a href="https://colab.research.google.com/github/debjit31/MNIST-Digit-Recognizer/blob/master/MNIST_Kaggle_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [97]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [98]:
train_path = '/content/drive/My Drive/digit-recognizer/train.csv'
train_data = pd.read_csv(train_path)

test_path = '/content/drive/My Drive/digit-recognizer/test.csv'
test_data = pd.read_csv(test_path)

In [99]:
test_data.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [100]:
train_data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [101]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [102]:
train_data.columns

Index(['label', 'pixel0', 'pixel1', 'pixel2', 'pixel3', 'pixel4', 'pixel5',
       'pixel6', 'pixel7', 'pixel8',
       ...
       'pixel774', 'pixel775', 'pixel776', 'pixel777', 'pixel778', 'pixel779',
       'pixel780', 'pixel781', 'pixel782', 'pixel783'],
      dtype='object', length=785)

In [123]:
y_train = train_data['label'].to_numpy()
print(type(y_train))
print(y_train.shape)
print(type(train_data['label']))

<class 'numpy.ndarray'>
(42000,)
<class 'pandas.core.series.Series'>


In [104]:
x_train = train_data.iloc[:, 1:].to_numpy()
print(x_train.shape)
print(type(x_train))

(42000, 784)
<class 'numpy.ndarray'>


In [105]:
x_tr = []
for i in range(len(x_train)):
  x_tr.append(x_train[i].reshape(28,28))
X_train = np.array(x_tr)

In [106]:
x_test = test_data.iloc[:, :].to_numpy()
x_te = []
for i in range(len(x_test)):
  x_te.append(x_test[i].reshape(28, 28))
X_test = np.array(x_te)


In [107]:
X_test.shape

(28000, 28, 28)

In [108]:
X_train.shape

(42000, 28, 28)

In [109]:
X_train = X_train / 255.0
X_test = X_test / 255.0

In [110]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, eopch, logs={}):
    if logs.get('accuracy') >= 0.99:
      print("\nReached 95% Accuracy,  Cancelling Training!!!\n")
      self.model.stop_training=True


def train_model():

  callback = MyCallback()
  model = tf.keras.models.Sequential()

  model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
  model.add(tf.keras.layers.Dense(1024, activation=tf.nn.relu))
  model.add(tf.keras.layers.Dense(10, activation=tf.nn.softmax))

  model.compile(optimizer = tf.optimizers.Adam(), 
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])
  
  model.fit(X_train, y_train, epochs = 1000, callbacks = [callback])

  return model


In [111]:
model = train_model()

Epoch 1/1000
1313/1313 [==============================] - 3s 3ms/step - loss: 0.2106 - accuracy: 0.9368
Epoch 2/1000
1313/1313 [==============================] - 3s 3ms/step - loss: 0.0831 - accuracy: 0.9745
Epoch 3/1000
1313/1313 [==============================] - 3s 3ms/step - loss: 0.0530 - accuracy: 0.9833
Epoch 4/1000
1313/1313 [==============================] - 3s 3ms/step - loss: 0.0353 - accuracy: 0.9883
Epoch 5/1000
1293/1313 [============================>.] - ETA: 0s - loss: 0.0291 - accuracy: 0.9902
Reached 95% Accuracy,  Cancelling Training!!!

1313/1313 [==============================] - 3s 3ms/step - loss: 0.0292 - accuracy: 0.9903


In [112]:
pred = model.predict(X_test)

In [113]:
pred[0]

array([1.1323200e-13, 2.3464272e-15, 1.0000000e+00, 8.4438324e-11,
       1.4296402e-13, 2.4832729e-13, 2.1084511e-13, 2.7024092e-09,
       2.3744973e-12, 4.6741263e-14], dtype=float32)

In [114]:
predictions=[]
for i in range(len(pred)):
  y = pred[i].tolist()
  predictions.append([i+1, y.index(max(y))])

In [115]:
predictions[-1]

[28000, 2]

In [129]:
predictions=np.array(predictions)
print(predictions.shape)
print(type(predictions))

(28000, 2)
<class 'numpy.ndarray'>


In [134]:
sub = pd.DataFrame(data=predictions, columns = ['ImageId', 'Label'])
sub.head(20)

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


In [133]:
sub.to_csv('MNIST Predictions.csv', index=False)